In [20]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import regex as re
import os
import seaborn as sns
from openpyxl import load_workbook
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
%reload_ext rpy2.ipython

In [21]:
os.chdir('/Users/arvind/Documents/Work/Zoho WorkDrive (Educational Initiatives Private Limited)/My Folders/Projects/CSF/Analysis/Code/DQA/')

In [4]:
def clean_scores(col_list, df):
    for col in col_list:
        df.loc[:, col].fillna(0, inplace=True)
        df.loc[:, col].replace(88,0,inplace=True)
        df.loc[:, col].replace('88',0,inplace=True)
        df.loc[:, col].replace(999,0,inplace=True)
        df.loc[:, col].replace('999',0,inplace=True)
        df.loc[:, col].replace('UNDEFINED', 0, inplace=True)
        df.loc[:, col].replace('SKIPPED', 0, inplace=True)
        df.loc[:, col].replace('.', 0, inplace=True)
        df.loc[:, col] = df.loc[:, col].astype('int64')
    return

<h1>
    Import raw data files
</h1>

In [5]:
#Importing all scored files
mp_literacy_scored = pd.read_excel("./2022_11_27_mp_literacy_total_scores.xlsx", index_col=[0])
mp_numeracy_scored = pd.read_excel("./2022_11_08_mp_numeracy_total_scores.xlsx", index_col=[0])
up_literacy_scored = pd.read_excel("./2022_11_27_up_literacy_total_scores.xlsx", index_col=[0])
up_numeracy_scored = pd.read_excel("./2022_11_08_up_numeracy_total_scores.xlsx", index_col=[0])

In [6]:
up_literacy_raw = pd.read_excel("./2022_11_27_up_raw_literacy_cleaned.xlsx", index_col=[0])
up_numeracy_raw = pd.read_excel("./2022_11_08_up_raw_numeracy_cleaned.xlsx", index_col=[0])
mp_literacy_raw = pd.read_excel("./2022_11_27_mp_raw_literacy_cleaned.xlsx", index_col=[0])
mp_numeracy_raw = pd.read_excel("./2022_11_08_mp_raw_numeracy_cleaned.xlsx", index_col=[0])

In [7]:
all(col in up_literacy_scored.columns for col in mp_literacy_scored.columns)
all(col in up_numeracy_scored.columns for col in mp_numeracy_scored.columns)
all(col in up_literacy_raw.columns for col in mp_literacy_raw.columns)
all(col in up_numeracy_raw.columns for col in mp_numeracy_raw.columns)

True

In [8]:
mp_dictation_validation = pd.read_excel("./mp_dictation_validation_final.xlsx", index_col=[0])
up_dictation_validation = pd.read_excel("./up_dictation_validation_final.xlsx", index_col=[0])

In [9]:
up_dictation_validation = up_dictation_validation.rename(columns={'literacy9a_total':'9a','literacy9b_total':'9b'})

In [10]:
mp_dictation_validation['9a'] = mp_dictation_validation['9a'].replace('Data not found',np.NaN)
mp_dictation_validation['9b'] = mp_dictation_validation['9b'].replace('Data not found',np.NaN)
mp_dictation_validation['9a'] = mp_dictation_validation['9a'].replace('Data not found ',np.NaN)
mp_dictation_validation['9b'] = mp_dictation_validation['9b'].replace('Data not found ',np.NaN)
mp_dictation_validation['9a'] = mp_dictation_validation['9a'].replace('Data repeated',np.NaN)
mp_dictation_validation['9b'] = mp_dictation_validation['9b'].replace('Data repeated',np.NaN)
up_dictation_validation['9a'] = up_dictation_validation['9a'].replace('Data not found',np.NaN)
up_dictation_validation['9b'] = up_dictation_validation['9b'].replace('Data not found',np.NaN)
up_dictation_validation['9a'] = up_dictation_validation['9a'].replace('Data not found ',np.NaN)
up_dictation_validation['9b'] = up_dictation_validation['9b'].replace('Data not found ',np.NaN)
up_dictation_validation['9a'] = up_dictation_validation['9a'].replace('Data repeated',np.NaN)
up_dictation_validation['9b'] = up_dictation_validation['9b'].replace('Data repeated',np.NaN)

In [11]:
up_dictation_validation['9a'].unique()

array([ 7.,  1.,  9.,  6., nan,  5.,  3., 10.,  8.,  4.,  0.,  2.])

In [12]:
mp_dict_val = mp_dictation_validation[['school_details.UDISE_cd_label',
 'SI_std_name',
 'student_age',
 '9a',
 '9b']]

mp_literacy_scored = pd.merge(mp_literacy_scored,mp_dict_val,\
                              how='left',on=['school_details.UDISE_cd_label',\
                                                'SI_std_name',\
                                                'student_age'])

In [13]:
mp_literacy_scored['literacy9a_total'] = mp_literacy_scored['9a'].combine_first(mp_literacy_scored['literacy9a_total'])
mp_literacy_scored['literacy9b_total'] = mp_literacy_scored['9b'].combine_first(mp_literacy_scored['literacy9b_total'])
mp_literacy_scored.drop(['9a', '9b'], axis=1, inplace=True)


In [14]:
up_dict_val = up_dictation_validation[['school_details.UDISE_cd_label',
 'SI_std_name',
 'student_age',
 '9a',
 '9b']]

up_literacy_scored = pd.merge(up_literacy_scored,up_dict_val,\
                              how='left',on=['school_details.UDISE_cd_label',\
                                                'SI_std_name',\
                                                'student_age'])

In [15]:
up_literacy_scored['literacy9a_total'] = up_literacy_scored['9a'].combine_first(up_literacy_scored['literacy9a_total'])
up_literacy_scored['literacy9b_total'] = up_literacy_scored['9b'].combine_first(up_literacy_scored['literacy9b_total'])
up_literacy_scored.drop(['9a', '9b'], axis=1, inplace=True)


In [16]:
mp_grouped = mp_numeracy_scored.groupby('school_details.UDISE_cd_label')['SI_std_name'].count().to_frame('cluster_size')

up_grouped = up_numeracy_scored.groupby('school_details.UDISE_cd_label')['SI_std_name'].count().to_frame('cluster_size')

In [17]:
mp_included_udise = mp_grouped[mp_grouped.cluster_size >=3].index.to_list()

up_included_udise = up_grouped[up_grouped.cluster_size >=5].index.to_list()

In [19]:
mp_included_udise

[23070107302,
 23070108204,
 23070110202,
 23070111205,
 23070111801,
 23070112804,
 23070116701,
 23070118402,
 23070300901,
 23070301001,
 23070302102,
 23070305001,
 23070306303,
 23070307501,
 23070307902,
 23070310802,
 23070312004,
 23070312503,
 23070313101,
 23070314203,
 23070318701,
 23070322901,
 23070328701,
 23070328801,
 23070402601,
 23070404002,
 23070407202,
 23070407803,
 23070409002,
 23070412701,
 23070412802,
 23070415404,
 23070417003,
 23070417004,
 23070419602,
 23070420004,
 23070431901,
 23070436601,
 23070437501,
 23070438101,
 23070604503,
 23070604604,
 23070605102,
 23070605301,
 23070605901,
 23070614201,
 23070616601,
 23070616901,
 23070617402,
 23070629301,
 23070629701,
 23070900101,
 23070905805,
 23070906102,
 23070908502,
 23070909203,
 23070909502,
 23070914601,
 23070916001,
 23070917301,
 23070932101,
 23070934801,
 23070938902,
 23210101301,
 23210111002,
 23210411401,
 23210507801,
 23210606601,
 23220402905,
 23220404305,
 23220405102,
 23220

In [220]:
mp_numeracy_scored = mp_numeracy_scored[mp_numeracy_scored['school_details.UDISE_cd_label'].isin(mp_included_udise)]

up_numeracy_scored = up_numeracy_scored[up_numeracy_scored['school_details.UDISE_cd_label'].isin(up_included_udise)]

In [221]:
mp_supervisors = ['Manohar', 'Jeevan', 'Vikram']

mp_irr_filter = mp_numeracy_scored.duplicated(subset=['school_details.UDISE_cd_label', 'SI_std_name','student_age'],keep=False)

mp_irr_assessments = mp_numeracy_scored[mp_irr_filter & mp_numeracy_scored.tabletUserName.isin(mp_supervisors)]

mp_numeracy_scored.drop(labels=mp_irr_assessments.index, inplace=True)

In [222]:
up_supervisors = ['Ambrish', 'Ashfak', 'Hina', 'Prrmshankar', 'Saurabh', 'Kuldeep']

up_irr_filter = up_numeracy_scored.duplicated(subset=['school_details.UDISE_cd_label', 'SI_std_name','student_age'],keep=False)

up_irr_assessments = up_numeracy_scored[up_irr_filter & up_numeracy_scored.tabletUserName.isin(up_supervisors)]

up_numeracy_scored.drop(labels=up_irr_assessments.index, inplace=True)

In [223]:
#Merging state files
literacy_scored = pd.concat([mp_literacy_scored,up_literacy_scored],join='inner')
numeracy_scored = pd.concat([mp_numeracy_scored,up_numeracy_scored],join='inner')

In [224]:
#Cleaning up state name
literacy_scored['school_details.State_label'] = literacy_scored['school_details.State_label'].replace('Madhya_Pradesh','Madhya Pradesh')
literacy_scored['school_details.State_label'] = literacy_scored['school_details.State_label'].replace('Uttar_Pradesh','Uttar Pradesh')
numeracy_scored['school_details.State_label'] = numeracy_scored['school_details.State_label'].replace('Madhya_Pradesh','Madhya Pradesh')
numeracy_scored['school_details.State_label'] = numeracy_scored['school_details.State_label'].replace('Uttar_Pradesh','Uttar Pradesh')                                                                                                    

In [225]:
#renaming variables for ease and use in R
literacy_scored = literacy_scored.rename(columns = {'school_details.State_label':'State',
                                  'school_details.District_label':'District',
                                  'school_details.Block_label':'Block',
                                  'school_details.School_label':'School',
                                  'school_details.UDISE_cd_label':'UDISE',
                                  'SI_std_name':'Name',
                                  'student_age':'Age',
                                  'student_gender':'Gender',
                                 'literacy1_total':'Listening_Comprehension',
                                 'literacy2_total':'Oral_Vocabulary',
                                 'literacy3_total':'Initial_Sound_Identification',
                                 'literacy4_ut_total':'Letter_Accuracy',
                                 'literacy4_tt_total':'Letter_Fluency_Count',
                                 'literacy4_tt_time_taken':'Letter_Fluency_Time',
                                 'literacy4_tt_cpm':'Letter_Fluency',
                                 'literacy5_ut_total':'Word_Accuracy',
                                 'literacy5_tt_total':'Word_Fluency_Count',
                                 'literacy5_tt_time_taken':'Word_Fluency_Time',
                                 'literacy5_tt_cpm':'Word_Fluency',
                                 'literacy6_total':'Non_Word_Fluency_Count',
                                 'literacy6_tt_time_taken':'Non_Word_Fluency_Time',
                                 'literacy6_tt_cpm':'Non_Word_Fluency',
                                 'literacy7_total':'Oral_Reading_Fluency_Count',
                                 'literacy7_tt_time_taken':'Oral_Reading_Fluency_Time',
                                 'literacy7_tt_cpm':'Oral_Reading_Fluency',
                                 'literacy8_reading_total':'Reading_Comprehension_Accuracy',
                                 'literacy8_comprehension_total':'Reading_Comprehension_Questions',
                                 'literacy9a_total':'Letter_Writing',
                                 'literacy9b_total':'Word_Writing'
                                                   })
numeracy_scored = numeracy_scored.rename(columns = {'school_details.State_label':'State',
                                  'school_details.District_label':'District',
                                  'school_details.Block_label':'Block',
                                  'school_details.School_label':'School',
                                  'school_details.UDISE_cd_label':'UDISE',
                                  'SI_std_name':'Name',
                                  'student_age':'Age',
                                  'student_gender':'Gender',
                                 'numeracy1_total':'Counting_Count',
                                 'numeracy1_tt_time_taken':'Counting_Time',
                                 'numeracy1_tt_cpm':'Counting',
                                 'numeracy2_ut_total':'Number_Recognition_Untimed',
                                 'numeracy2_tt_total':'Number_Recognition_Timed_Count',
                                 'numeracy2_tt_time_taken':'Number_Recognition_Timed_Time',
                                 'numeracy2_tt_cpm':'Number_Recognition_Timed',
                                 'numeracy3_total':'Number_Comparison',
                                 'numeracy4_total':'Counting_in_Bundles',
                                 'numeracy5_total':'Missing_Numbers',
                                 'numeracy6_total':'Addition',
                                 'numeracy7_total':'Subtraction',
                                 'numeracy8_total':'Word_Problems',
                                 'numeracy9a_total':'Shape_Recognition_Circle',
                                 'numeracy9b_total':'Shape_Recognition_Square'                
                                                   })

In [226]:
literacy_scored['id'] = literacy_scored.index
numeracy_scored['id'] = numeracy_scored.index

In [227]:
main_var = [
 'id',
 'tabletUserName',
 'assessment_date',
 'State',
 'District',
 'Block',
 'School',
 'UDISE',
 'Name',
 'Age',
 'Gender']

In [228]:
all_var_lit = literacy_scored.columns.to_list()

In [229]:
all_var_num = numeracy_scored.columns.to_list()

In [230]:
rest_var_lit = [x for x in all_var_lit if x not in main_var]

In [231]:
rest_var_num = [x for x in all_var_num if x not in main_var]

In [232]:
literacy_scored_long = pd.melt(literacy_scored,id_vars = main_var,value_vars = rest_var_lit)

In [233]:
numeracy_scored_long = pd.melt(numeracy_scored,id_vars = main_var,value_vars = rest_var_num)

In [234]:
literacy_scored_long['Type'] = 'literacy'

In [235]:
numeracy_scored_long['Type'] = 'numeracy'

In [236]:
lat_long = pd.read_csv("../Results/csf_lat_long.csv")

/var/folders/0c/r0hw428532s7bttp9my0ym_00000gn/T/ipykernel_66236/2693788372.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  lat_long = pd.read_csv("../Results/csf_lat_long.csv")


In [237]:
scored_long = pd.concat([literacy_scored_long,numeracy_scored_long],join='inner')

In [238]:
scored_long = scored_long.merge(lat_long[['udise_sch_code','latitude','longitude']]
                                                  ,left_on="UDISE"
                                                  ,right_on="udise_sch_code"
                                                  ,how='left'
                                                  ).drop(columns=['udise_sch_code'])

In [239]:
scored_long['cohort'] = np.where(scored_long['Block'] == 'Sewapuri','T1'
                                          ,np.where(scored_long['District'] == 'Varanasi','T2'
                                              ,np.where(scored_long['District'] == 'Sehore','T'
                                                        ,np.where(scored_long['District'] == 'Shajapur','T'
                                                                 ,'C'))))
                                                    

In [240]:
scored_long.to_csv("../Results/scored_long.csv")

In [16]:
scored_long = pd.read_csv("../Results/scored_long.csv")

/var/folders/0c/r0hw428532s7bttp9my0ym_00000gn/T/ipykernel_66236/1315116701.py:1: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  scored_long = pd.read_csv("../Results/scored_long.csv")


In [253]:
scored_long.head()

,id,tabletUserName,assessment_date,State,District,Block,School,UDISE,Name,Age,Gender,variable,value,Type,latitude,longitude,cohort
0,0,Jeevan,2022-09-17,Madhya Pradesh,Shajapur,M. Barodiya,GIRLS P.S. PALSAWAD,23220413303,Bhumika,5.0,Female,Listening_Comprehension,3.0,literacy,23.372602,76.558021,T
1,1,Kajal,2022-09-23,Madhya Pradesh,Vidisha,Gyaraspur,PS NIPANIYA,23310211501,pavan kushwah,6.0,Male,Listening_Comprehension,4.0,literacy,23.607171,77.942319,C
2,2,Rahnuma,2022-09-16,Madhya Pradesh,Shajapur,Shajapur,P.S. RATANPURA,23220603401,lakshmi gurjar,5.0,Female,Listening_Comprehension,1.0,literacy,23.240781,76.469734,T
3,3,Vaibhav,2022-09-23,Madhya Pradesh,Shajapur,Shajapur,BOYS M.S. MAHUPURA SHAJAPUR,23220605849,radhika rodu,6.0,Female,Listening_Comprehension,0.0,literacy,23.425583,76.284243,T
4,4,Nanda,2022-09-16,Madhya Pradesh,Shajapur,Shajapur,NEW M.S. DILLOD,23220619202,Subham Ramesh,6.0,Male,Listening_Comprehension,0.0,literacy,23.362712,76.269730,T


In [254]:
scored_long_lit = scored_long[scored_long["Type"]=='literacy']
scored_long_num = scored_long[scored_long["Type"]=='numeracy']

In [255]:
scored_wide_lit = pd.pivot(scored_long_lit[['id','Name','UDISE','cohort','State','variable','value']]
                        ,index=['id','Name','UDISE','cohort','State']
                        ,columns = 'variable'
                        , values= 'value')

scored_wide_num = pd.pivot(scored_long_num[['id','Name','UDISE','cohort','State','variable','value']]
                        ,index=['id','Name','UDISE','cohort','State']
                        ,columns = 'variable'
                        , values= 'value')



/opt/anaconda3/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


In [256]:
scored_wide_lit.to_csv("../Results/scored_wide_lit.csv")
scored_wide_num.to_csv("../Results/scored_wide_num.csv")

In [18]:
# dict = {'Uttar Pradesh':[5,['literacy','numeracy']],'Madhya Pradesh':[3,['literacy','numeracy']]}

In [19]:
# for key, value in dict.items():
#     for type in value[1]:
#         df = scored_long[(scored_long["Type"]== type) & (scored_long["State"]==key)].\
#         groupby(['UDISE'],axis=0)['Name'].\
#         nunique().to_frame('Count')
#         included = df[df['Count'] >= value[0]].index.to_list()
#         scored_long = scored_long[scored_long['UDISE'].isin(included)]

In [62]:
mp_grouped = mp_numeracy_raw.groupby('school_details.UDISE_cd_label')['SI_std_name'].count().to_frame('cluster_size')

up_grouped = up_numeracy_raw.groupby('school_details.UDISE_cd_label')['SI_std_name'].count().to_frame('cluster_size')

In [63]:
mp_included_udise = mp_grouped[mp_grouped.cluster_size >=3].index.to_list()

up_included_udise = up_grouped[up_grouped.cluster_size >=5].index.to_list()

In [64]:
mp_numeracy_raw = mp_numeracy_raw[mp_numeracy_raw['school_details.UDISE_cd_label'].isin(mp_included_udise)]

up_numeracy_raw = up_numeracy_raw[up_numeracy_raw['school_details.UDISE_cd_label'].isin(up_included_udise)]

In [65]:
mp_supervisors = ['Manohar', 'Jeevan', 'Vikram']

mp_irr_filter = mp_numeracy_raw.duplicated(subset=['school_details.UDISE_cd_label', 'SI_std_name','student_age'],keep=False)

mp_irr_assessments = mp_numeracy_raw[mp_irr_filter & mp_numeracy_raw.tabletUserName.isin(mp_supervisors)]

mp_numeracy_raw.drop(labels=mp_irr_assessments.index, inplace=True)

In [66]:
up_supervisors = ['Ambrish', 'Ashfak', 'Hina', 'Prrmshankar', 'Saurabh', 'Kuldeep']

up_irr_filter = up_numeracy_raw.duplicated(subset=['school_details.UDISE_cd_label', 'SI_std_name','student_age'],keep=False)

up_irr_assessments = up_numeracy_raw[up_irr_filter & up_numeracy_raw.tabletUserName.isin(up_supervisors)]

up_numeracy_raw.drop(labels=up_irr_assessments.index, inplace=True)

In [67]:
#Merging state files for item analysis
literacy_raw = pd.concat([mp_literacy_raw,up_literacy_raw],join='inner')
numeracy_raw = pd.concat([mp_numeracy_raw,up_numeracy_raw],join='inner')

In [68]:
literacy_raw['id']= list(literacy_raw.index)
numeracy_raw['id']= list(numeracy_raw.index)

/var/folders/0c/r0hw428532s7bttp9my0ym_00000gn/T/ipykernel_66236/13276588.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  numeracy_raw['id']= list(numeracy_raw.index)


In [69]:
#renaming variables for ease and use in R
literacy_raw = literacy_raw.rename(columns = {'school_details.State_label':'State',
                                  'school_details.District_label':'District',
                                  'school_details.Block_label':'Block',
                                  'school_details.School_label':'School',
                                  'school_details.UDISE_cd_label':'UDISE',
                                  'SI_std_name':'Name',
                                  'student_age':'Age',
                                  'student_gender':'Gender'
                                                   })
numeracy_raw = numeracy_raw.rename(columns = {'school_details.State_label':'State',
                                  'school_details.District_label':'District',
                                  'school_details.Block_label':'Block',
                                  'school_details.School_label':'School',
                                  'school_details.UDISE_cd_label':'UDISE',
                                  'SI_std_name':'Name',
                                  'student_age':'Age',
                                  'student_gender':'Gender'               
                                                   })

In [70]:
literacy_subtask_var = [col for col in literacy_raw.columns 
                        if re.search(r'literacy1_q\d$', col)
                        or re.search(r'literacy2_q\d+$', col)
                        or re.search(r'literacy3_q+', col)
                        or re.search(r'literacy4_ut_grid_\d*$', col)
                        or re.search(r'literacy4_tt_grid_\d*$', col)
                        or re.search(r'literacy5_ut_grid_\d*$', col)
                        or re.search(r'literacy5_tt_grid_\d*$', col)
                        or re.search(r'literacy6_tt_grid_\d*$', col)
                        or re.search(r'literacy7_tt_grid_\d*$', col)
                        or re.search(r'literacy8_ut_grid_\d*$', col)
                        or re.search(r'literacy8_ut_q\d*$', col)
                        or re.search(r'literacy9a_ut_grid_\d*$', col)
                        or re.search(r'literacy9b_ut_grid_\d*$', col)  
                       ]

In [71]:
numeracy_subtask_var = [col for col in numeracy_raw.columns
                       if re.search(r'numeracy1_tt_grid_\d*$', col)
                       or re.search(r'numeracy2_ut_grid_\d*$', col)
                       or re.search(r'numeracy2_tt_grid_\d*$', col)
                       or re.search(r'numeracy3\w*', col)
                       or re.search(r'numeracy4_ut_q\d*$', col)
                       or re.search(r'numeracy5_ut_q\d*$', col)
                       or re.search(r'numeracy6_ut_q\d*$', col)
                       or re.search(r'numeracy7_ut_q\d*$', col)
                       or re.search(r'numeracy8_ut_q\d*$', col)
                       or re.search(r'numeracy9a_ut_grid_\d$', col)
                       or re.search(r'numeracy9b_ut_grid_\d$', col)
                       ]

In [72]:
literacy_subtask_long = pd.melt(literacy_raw,id_vars = main_var,value_vars = literacy_subtask_var)

In [73]:
list_clean = ['value']

In [74]:
clean_scores(list_clean, literacy_subtask_long)

In [75]:
literacy_subtask_long[['subtask','item']] = literacy_subtask_long['variable'].str.split('_',1,expand=True)

In [76]:
literacy_subtask_long['flag'] = literacy_subtask_long['item'].str[:2]

In [77]:
literacy_subtask_long['timed_untimed'] = np.where(literacy_subtask_long['flag']=='tt','tt','ut')

In [78]:
literacy_subtask_long.to_csv("../Results/literacy_subtask_scored.csv")

In [79]:
numeracy_subtask_long = pd.melt(numeracy_raw, id_vars = main_var, value_vars = numeracy_subtask_var)

In [80]:
clean_scores(list_clean, numeracy_subtask_long)

In [81]:
numeracy_subtask_long[['subtask','item']] = numeracy_subtask_long['variable'].str.split('_',1,expand=True)

In [82]:
numeracy_subtask_long['flag'] = numeracy_subtask_long['item'].str[:2]

In [83]:
numeracy_subtask_long['timed_untimed'] = np.where(numeracy_subtask_long['flag']=='tt','tt','ut')

In [84]:
numeracy_subtask_long.to_csv("../Results/numeracy_subtask_scored.csv")

In [85]:
literacy_subtask_long['Type'] = 'literacy'

In [86]:
numeracy_subtask_long['Type'] = 'numeracy'

In [87]:
subtask_long = pd.concat([literacy_subtask_long,numeracy_subtask_long],join='inner')

In [88]:
numeracy_subtask_long.to_csv("../Results/numeracy_subtask_scored.csv")

In [89]:
variable_item_mapping = pd.read_csv("../Results/variable_item_mapping.csv")

In [90]:
subtask_long = subtask_long.merge(variable_item_mapping[['Original_Variable','Group_Variable','Actual_Item']]
                                                  ,left_on="variable"
                                                  ,right_on="Original_Variable"
                                                  ,how='left'
                                                  ).drop(columns=['Original_Variable'])

In [91]:
subtask_long.to_csv("../Results/subtask_scored_long.csv")

In [92]:
lst = literacy_subtask_long['variable'].unique()

In [93]:
dfr = pd.DataFrame(lst)

In [94]:
dfr.to_csv("../Results/literacy_var.csv")

In [96]:
subtask_long.shape

(2080260, 20)

In [245]:
subtask_long_lit = subtask_long[subtask_long["Type"]=='literacy']

In [246]:
subtask_long_lit.head()

,id,tabletUserName,assessment_date,State,District,Block,School,UDISE,Name,Age,Gender,variable,value,subtask,item,flag,timed_untimed,Type,Group_Variable,Actual_Item
0,0050c62f-02ba-4cc6-bb95-bf76e33e5069,Jeevan,2022-09-17,Madhya Pradesh,Shajapur,M. Barodiya,GIRLS P.S. PALSAWAD,23220413303,Bhumika,5.0,Female,literacy1_q1,1,literacy1,q1,q1,ut,literacy,listening_comprehension,1. इस कहानी में कौन-कौन से जानवर है?
1,00d4fdf3-5016-4037-a386-148bf838ff90,Kajal,2022-09-23,Madhya Pradesh,Vidisha,Gyaraspur,PS NIPANIYA,23310211501,pavan kushwah,6.0,Male,literacy1_q1,1,literacy1,q1,q1,ut,literacy,listening_comprehension,1. इस कहानी में कौन-कौन से जानवर है?
2,010fada9-996a-49c2-a8af-bdb6c5c688a3,Rahnuma,2022-09-16,Madhya Pradesh,Shajapur,Shajapur,P.S. RATANPURA,23220603401,lakshmi gurjar,5.0,Female,literacy1_q1,0,literacy1,q1,q1,ut,literacy,listening_comprehension,1. इस कहानी में कौन-कौन से जानवर है?
3,015e4b93-016a-457c-8987-65a7a048b051,Vaibhav,2022-09-23,Madhya Pradesh,Shajapur,Shajapur,BOYS M.S. MAHUPURA SHAJAPUR,23220605849,radhika rodu,6.0,Female,literacy1_q1,0,literacy1,q1,q1,ut,literacy,listening_comprehension,1. इस कहानी में कौन-कौन से जानवर है?
4,0170015d-21f2-4342-a66e-bd9ad6f49162,Nanda,2022-09-16,Madhya Pradesh,Shajapur,Shajapur,NEW M.S. DILLOD,23220619202,Subham Ramesh,6.0,Male,literacy1_q1,0,literacy1,q1,q1,ut,literacy,listening_comprehension,1. इस कहानी में कौन-कौन से जानवर है?


In [244]:
subtask_wide = pd.pivot(subtask_long_lit[['id','Name','UDISE','cohort','variable','value']]
                        ,index=['id','Name','UDISE','cohort']
                        ,columns = 'variable'
                        , values= 'value')

KeyError: "['cohort'] not in index"

In [243]:
subtask_wide.to_csv("../Results/subtask_wide_lit.csv")

In [22]:
subtask_long = pd.read_csv("../Results/subtask_scored_long.csv")

/var/folders/0c/r0hw428532s7bttp9my0ym_00000gn/T/ipykernel_70223/2759883781.py:1: DtypeWarning: Columns (1,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  subtask_long = pd.read_csv("../Results/subtask_scored_long.csv")


In [23]:
subtask_long_num = subtask_long[subtask_long["Type"]=='numeracy']

In [25]:
subtask_long_num.head()

,Unnamed: 0,id,tabletUserName,assessment_date,State,District,Block,School,UDISE,Name,Age,Gender,variable,value,subtask,item,flag,timed_untimed,Type,Group_Variable,Actual_Item
1572500,1572500,0,Jeevan,2022-09-17,Madhya Pradesh,Shajapur,M. Barodiya,GIRLS P.S. PALSAWAD,23220413303,Bhumika,5.0,Female,numeracy1_tt_grid_1,1,numeracy1,tt_grid_1,tt,tt,numeracy,NaN,NaN
1572501,1572501,1,Kajal,2022-09-23,Madhya Pradesh,Vidisha,Gyaraspur,PS NIPANIYA,23310211501,pavan kushwah,6.0,Male,numeracy1_tt_grid_1,1,numeracy1,tt_grid_1,tt,tt,numeracy,NaN,NaN
1572502,1572502,2,Rahnuma,2022-09-16,Madhya Pradesh,Shajapur,Shajapur,P.S. RATANPURA,23220603401,lakshmi gurjar,5.0,Female,numeracy1_tt_grid_1,1,numeracy1,tt_grid_1,tt,tt,numeracy,NaN,NaN
1572503,1572503,3,Vaibhav,2022-09-23,Madhya Pradesh,Shajapur,Shajapur,BOYS M.S. MAHUPURA SHAJAPUR,23220605849,radhika rodu,6.0,Female,numeracy1_tt_grid_1,1,numeracy1,tt_grid_1,tt,tt,numeracy,NaN,NaN
1572504,1572504,4,Nanda,2022-09-16,Madhya Pradesh,Shajapur,Shajapur,NEW M.S. DILLOD,23220619202,Subham Ramesh,6.0,Male,numeracy1_tt_grid_1,1,numeracy1,tt_grid_1,tt,tt,numeracy,NaN,NaN


In [26]:
subtask_wide_num = pd.pivot(subtask_long_num[['id','Name','UDISE','variable','value']]
                        ,index=['id','Name','UDISE',]
                        ,columns = 'variable'
                        , values= 'value')

In [27]:
subtask_wide_num.to_csv("../Results/subtask_wide_num.csv")

In [ ]:
%%R 

install.packages("ltm")

# require(dplyr)
# require(Hmisc)
# require(sqldf)
# require(ltm)
# require(mirt)
# require(irtoys)
# require(readr)
# require(tidyverse)

# IRTmodel_lit = ltm(subtask_wide~z1,IRT.param=TRUE)
# IRTcoef = coef(IRTmodel_lit)
# irtcf = as.data.frame(IRTcoef)
# irtcf = rownames_to_column(irtcf, var = "Item")
# factor_scores = factor.scores(IRTmodel_lit)
# person_fit = person.fit(IRTmodel_lit)
# item_fit = item.fit(IRTmodel_lit)

# import rpy2.robjects as ro
# from rpy2.robjects.packages import importr
# from rpy2.robjects import pandas2ri
# from rpy2.robjects.conversion import localconverter
# with localconverter(ro.default_converter + pandas2ri.converter):
#   dt = ro.conversion.rpy2py(irtcf)

In [227]:
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
with localconverter(ro.default_converter + pandas2ri.converter):
  dt = ro.conversion.rpy2py(irtcf)

NameError: name 'irtcf' is not defined

In [14]:
# Check for gender distribution
# literacy_scored.groupby(['State','Gender'])['id'].count()

In [15]:
# # check for student responses per school greater than 12
# chk1 = pd.DataFrame(literacy_scored.groupby(['UDISE'])['id'].count())
# chk1['UDISE'] = chk1.index
# chk1 = pd.DataFrame(chk1.groupby(['id'])['UDISE'].count())
# chk2 = pd.DataFrame(numeracy_scored.groupby(['UDISE'])['id'].count())
# chk2['UDISE'] = chk2.index
# chk2 = pd.DataFrame(chk2.groupby(['id'])['UDISE'].count())
# chk1.merge(chk2,how='left',on='id')

In [16]:
# chk = pd.DataFrame(literacy_scored.groupby(['UDISE','Name','Age','Gender'])['id'].count())
# chk['UDISE'] = chk.index
# chk.groupby(['id'])['UDISE'].count()

In [17]:
# literacy_scored[literacy_scored['literacy9b_total']>7].groupby(['State','District'])['Name'].count()